In [1]:
import json
import sqlite3
import numpy as np
import pandas as pd
import rasterio
import geopandas as gpd

In [2]:
satellite_types = [
    {
        "satellite_type_id": 0,
        "name": "passive",
    },
    {
        "satellite_type_id": 1,
        "name": "active",
    },
]

In [3]:
satellites = [
    {
        "satellite_id": 0,
        "name": "Sentinel 2",
        "satellite_type": 0,
    },
    {
        "satellite_id": 1,
        "name": "Landsat 8",
        "satellite_type": 0,
    },
    {
        "satellite_id": 2,
        "name": "Landsat 7",
        "satellite_type": 0,
    },
    {
        "satellite_id": 3,
        "name": "Landsat 5",
        "satellite_type": 0,
    },
]

In [4]:
sources = [
    {
        "source_id": 0,
        "name": "Google Earth Engine",
        "link": "https://earthengine.google.com/",
    },
    {
        "source_id": 1,
        "name": "USGS",
        "link": "https://earthexplorer.usgs.gov/",
    },
    {
        "source_id": 2,
        "name": "Copernicus",
        "link": "https://browser.dataspace.copernicus.eu/",
    },
]

In [5]:
processing_methods = [
    {
        "processing_method_id": 0,
        "name": "noise cancelling",
        "description": "",
    },
    {
        "processing_method_id": 1,
        "name": "normalization",
        "description": "",
    },
    {
        "processing_method_id": 2,
        "name": "reprojection",
        "description": "",
    },
        {
        "processing_method_id": 3,
        "name": "scaling",
        "description": "",
    },
    {
        "processing_method_id": 4,
        "name": "spectral correction",
        "description": "",
    },
    {
        "processing_method_id": 5,
        "name": "cropping",
        "description": "",
    },
]

In [6]:
channels = []

# Sentinel 2
channels += [
    {
        "channel_id": 0,
        "satellite_id": 0,
        "name": "Band 1",
        "description": "Coastal aerosol",
        "central_wavelength": 0.443,
        "pixel_size": 60,
    },
    {
        "channel_id": 1,
        "satellite_id": 0,
        "name": "Band 2",
        "description": "Blue",
        "central_wavelength": 0.490,
        "pixel_size": 10,
    },
    {
        "channel_id": 2,
        "satellite_id": 0,
        "name": "Band 3",
        "description": "Green",
        "central_wavelength": 0.56,
        "pixel_size": 10,
    },
    {
        "channel_id": 3,
        "satellite_id": 0,
        "name": "Band 4",
        "description": "Red",
        "central_wavelength": 0.665,
        "pixel_size": 10,
    },
    {
        "channel_id": 4,
        "satellite_id": 0,
        "name": "Band 5",
        "description": "Vegetation red edge",
        "central_wavelength": 0.705,
        "pixel_size": 20,
    },
    {
        "channel_id": 5,
        "satellite_id": 0,
        "name": "Band 6",
        "description": "Vegetation red edge",
        "central_wavelength": 0.74,
        "pixel_size": 20,
    },
    {
        "channel_id": 6,
        "satellite_id": 0,
        "name": "Band 7",
        "description": "Vegetation red edge",
        "central_wavelength": 0.783,
        "pixel_size": 20,
    },
    {
        "channel_id": 7,
        "satellite_id": 0,
        "name": "Band 8",
        "description": "NIR",
        "central_wavelength": 0.842,
        "pixel_size": 10,
    },
    {
        "channel_id": 8,
        "satellite_id": 0,
        "name": "Band 8A",
        "description": "Vegetation red edge",
        "central_wavelength": 0.865,
        "pixel_size": 20,
    },
    {
        "channel_id": 9,
        "satellite_id": 0,
        "name": "Band 9",
        "description": "Water vapour",
        "central_wavelength": 0.945,
        "pixel_size": 60,
    },
    {
        "channel_id": 10,
        "satellite_id": 0,
        "name": "Band 10",
        "description": "SWIR - Cirrus",
        "central_wavelength": 1.375,
        "pixel_size": 60,
    },
    {
        "channel_id": 11,
        "satellite_id": 0,
        "name": "Band 11",
        "description": "SWIR",
        "central_wavelength": 1.610,
        "pixel_size": 20,
    },
    {
        "channel_id": 12,
        "satellite_id": 0,
        "name": "Band 12",
        "description": "SWIR",
        "central_wavelength": 2.19,
        "pixel_size": 20,
    },
]

# Landsat 8
channels += [
    {
        "channel_id": 13,
        "satellite_id": 1,
        "name": "Band 1",
        "description": "Coastal aerosol",
        "central_wavelength": 0.44,
        "pixel_size": 30,
    },
    {
        "channel_id": 14,
        "satellite_id": 1,
        "name": "Band 2",
        "description": "Blue",
        "central_wavelength": 0.48,
        "pixel_size": 30,
    },
    {
        "channel_id": 15,
        "satellite_id": 1,
        "name": "Band 3",
        "description": "Green",
        "central_wavelength": 0.56,
        "pixel_size": 30,
    },
    {
        "channel_id": 16,
        "satellite_id": 1,
        "name": "Band 4",
        "description": "Red",
        "central_wavelength": 0.655,
        "pixel_size": 30,
    },
    {
        "channel_id": 17,
        "satellite_id": 1,
        "name": "Band 5",
        "description": "NIR",
        "central_wavelength": 0.865,
        "pixel_size": 30,
    },
    {
        "channel_id": 18,
        "satellite_id": 1,
        "name": "Band 6",
        "description": "SWIR 1",
        "central_wavelength": 1.61,
        "pixel_size": 30,
    },
    {
        "channel_id": 19,
        "satellite_id": 1,
        "name": "Band 7",
        "description": "SWIR 2",
        "central_wavelength": 2.2,
        "pixel_size": 30,
    },
    {
        "channel_id": 20,
        "satellite_id": 1,
        "name": "Band 8",
        "description": "Panchromatic",
        "central_wavelength": 0.59,
        "pixel_size": 15,
    },
    {
        "channel_id": 21,
        "satellite_id": 1,
        "name": "Band 9",
        "description": "Cirrus",
        "central_wavelength": 1.37,
        "pixel_size": 30,
    },
    {
        "channel_id": 22,
        "satellite_id": 1,
        "name": "Band 10",
        "description": "TIRS 1",
        "central_wavelength": 10.895,
        "pixel_size": 100,
    },
    {
        "channel_id": 23,
        "satellite_id": 1,
        "name": "Band 11",
        "description": "TIRS 2",
        "central_wavelength": 12.005,
        "pixel_size": 100,
    },
]

In [7]:
object_types = [
    {
        "object_type_id": 0,
        "name": "quarry",
    },
]

In [8]:
minerals = [
    {
        "mineral_id": 0,
        "name": "granodiorite"
    }
]

In [9]:
objects = [
    {
        "object_id": 0,
        "object_type_id": 0,
        "name": "korfovskiy",
        "shape_location": r"example\shapes\korfovskiy_quarry_20240717.shp",
        "description": "",
    },
]

object_minerals = [
    {
        "object_id": 0,
        "mineral_id": 0,
    },
]

In [10]:
report_types = [
    {
        "report_type_id": 0,
        "name": "сhanges in the area of ​​overgrowth",
        "description": "Indicators of the area of ​​overgrowth of the territory over time",
    },
]

In [11]:
location = r"example\shapes\korfovskiy_aoi_epsg32653.shp"
gdf = gpd.read_file(location)
minx, miny, maxx, maxy = gdf.total_bounds
areas_of_interest = [
    {
        "area_of_interest_id": 0,
        "coordinate_x": minx,
        "coordinate_y": maxx,
        "width": maxx - minx,
        "height": maxy - miny,
        "crs": str(gdf.crs),
        "shape_location": location,
        "description": "",
    },
]

object_areas = [
    {
        "area_of_interest_id": 0,
        "object_id": 0,
    },
]

reports = [
    {
        "report_id": 0,
        "area_of_interest_id": 0,
        "report_type": 0,
        "path": r"example/reports/Исследование изменения площади зарастания карьера Корфовского месторождения с 2004 по 2023 года.csv",
    },
]

In [12]:
location = r"example\raw\S2H_korfovskiy_20240717.tif"
with rasterio.open(location, "r") as f:
    img = f.read()
    meta = f.meta.copy()

raw_images = [
    {
        "raw_image_id": 0,
        "satellite_id": 0,
        "source_id": 0,
        "coordinate_x": meta["transform"].to_gdal()[0],
        "coordinate_y": meta["transform"].to_gdal()[3],
        "width": meta["width"],
        "height": meta["height"],
        "date": "20240717",
        "crs": str(meta["crs"]),
        "location": location,
    },
]

raw_image_channels = []
for i, channel_id in enumerate([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]):
    raw_image_channels += [
        {
            "raw_image_id": 0,
            "channel_id": channel_id,
            "position": i,
        },
    ]

In [13]:
location = r"example\processed\sentinel2_korfovskiy_20240717.tif"
with rasterio.open(location, "r") as f:
    img = f.read()
    meta = f.meta.copy()

processed_images = [
    {
        "processed_image_id": 0,
        "coordinate_x": meta["transform"].to_gdal()[0],
        "coordinate_y": meta["transform"].to_gdal()[3],
        "width": meta["width"],
        "height": meta["height"],
        "crs": str(meta["crs"]),
        "location": location,
        "pixel_size": meta["transform"][0],
    },
]

raw_processed_images = [
    {
        "raw_image_id": 0,
        "processed_image_id": 0,
    },
]

processed_image_channels = []
for i, channel_id in enumerate([1, 2, 3, 4, 5, 6, 7, 11, 12]):
    processed_image_channels += [
        {
            "processed_image_id": 0,
            "channel_id": channel_id,
            "position": i,
        },
    ]

processed_image_methods = []
for i, method in enumerate([0, 1, 2, 3]):
    processed_image_methods += [
        {
            "processed_image_id": 0,
            "processing_method_id": method,
            "sequence": i,
        },
    ]

processed_image_areas = [
    {
        "processed_image_id": 0,
        "area_of_interest_id": 0,
    },
]

### Добавление данных в БД

In [ ]:
db = r"db.sqlite"

con = sqlite3.connect(db)
cursor = con.cursor()

# Добавляемые таблицы
tables = {
    # "satellite_types": satellite_types,
    # "satellites": satellites,
    # "sources": sources,
    # "processing_methods": processing_methods,
    # "channels": channels,
    # "object_types": object_types,
    # "minerals": minerals,
    # "objects": objects,
    # "object_minerals": object_minerals,
    # "report_types": report_types,
    # "areas_of_interest": areas_of_interest,
    # "object_areas": object_areas,
    # "reports": reports,
    # "raw_images": raw_images,
    # "raw_image_channels": raw_image_channels,
    # "raw_processed_images": raw_processed_images,
    # "processed_images": processed_images,
    # "processed_image_channels": processed_image_channels,
    # "processed_image_methods": processed_image_methods,
    # "processed_image_areas": processed_image_areas,
}

for table_name, table in tables.items():
    check = cursor.execute(f"SELECT * FROM {table_name};")
    check = cursor.fetchall()
    con.commit()

    if not check:
        for x in table:
            try:
                qu = "INSERT INTO {0} ({1}) VALUES ({2});"\
                    .format(table_name,
                            ', '.join(map(str, x.keys())),
                            ', '.join(['\'{0}\''.format(i) if isinstance(i, str) else str(i) for i in x.values()]))
                # print(qu)
                cursor.execute(qu)
            except Exception as e:
                print(e)

    con.commit()
con.close()